In [3]:
# ruff: noqa: F401
import math
import sys
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    starmap,
    tee,
)
from math import comb, factorial, log, sqrt
from operator import add, and_, contains, indexOf, itemgetter, neg, or_, xor
from pprint import pprint
from typing import Callable, Iterable, List, Tuple

import matplotlib.pyplot as plt
import numpy as np

import arrays
import graphs
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import strings
import trees
from parsing import parse_arithmetic_expression
from sequences import find_if

for m in (
    arrays,
    trees,
    lists,
    graphs,
    parsing,
    search,
    strings,
    sets,
    sequences,
    matrix,
    mathematics,
):
    reload(m)

print()


In [21]:
def format_binop_expression(e: str | tuple) -> str:
    "Format expressions consisting of (<operator>, <arg1>, <arg2>)."
    if isinstance(e, str):
        return e
    # This is a depth first, iterative algorithm using two stacks.
    s: List[tuple] = []
    v: List[str | tuple] = []
    n: tuple | str = e
    # Flatten an expression.
    f = lambda v: v if isinstance(v, str) else v[1]

    def fop(op : str, a : str | tuple, b : str | tuple) -> tuple:
        "Format an operation adding parantheses to the arguments if needed."
        def p(v):
            if isinstance(v, str) or op == "+" or op in "*-" and v[0] in "*/":
                return f(v)
            return "(" + f(v) + ")"
        # Returns the operator and the string representation.
        # We keep the operator so it is easier to know the type of the expression later.
        return op, f(a) + op + p(b)

    while s or n:
        # Repeatedly, descend left from node.
        while isinstance(n, tuple):
            # If this is on the stack, we consume the node.
            s.append(n)
            # If this is on the stack, we descend right.
            s.append(n)
            n = n[1]
        if n is not None:
            v.append(n)
        n = s.pop()
        if s and s[-1] == n:
            # Descend right from node.
            n = n[2]
        else:
            # Consume this node and
            # generate intermediate values.
            rv, lv = v.pop(), v.pop()
            v.append(fop(n[0], lv, rv))
            n = None
    return f(v[0])


In [22]:
e = parse_arithmetic_expression("a-b-c")
print(e)
format_binop_expression(e)


('-', ('-', 'a', 'b'), 'c')


'a-b-c'

In [23]:
e = parse_arithmetic_expression("A-(B-C)")
print(e)
format_binop_expression(e)


('-', 'A', ('-', 'B', 'C'))


'A-(B-C)'

In [32]:
e = parse_arithmetic_expression("A+(B+C)")
print(e)
format_binop_expression(e)


('+', 'A', ('+', 'B', 'C'))


'A+B+C'

In [24]:
e = parse_arithmetic_expression("A+B*C")
print(e)
format_binop_expression(e)


('+', 'A', ('*', 'B', 'C'))


'A+B*C'

In [25]:
e = parse_arithmetic_expression("(A/(B*C))")
print(e)
format_binop_expression(e)


('/', 'A', ('*', 'B', 'C'))


'A/(B*C)'

In [26]:
e = parse_arithmetic_expression("A/(B/C)")
print(e)
format_binop_expression(e)


('/', 'A', ('/', 'B', 'C'))


'A/(B/C)'

In [27]:
e = parse_arithmetic_expression("X/Y/Z/(A+B)")
print(e)
format_binop_expression(e)


('/', ('/', ('/', 'X', 'Y'), 'Z'), ('+', 'A', 'B'))


'X/Y/Z/(A+B)'

In [28]:
e = parse_arithmetic_expression("X/Y/(Z/(A+B))")
print(e)
format_binop_expression(e)


('/', ('/', 'X', 'Y'), ('/', 'Z', ('+', 'A', 'B')))


'X/Y/(Z/(A+B))'

In [31]:
e = parse_arithmetic_expression("X*Y*(Z*(A+B))")
print(e)
format_binop_expression(e)


('*', ('*', 'X', 'Y'), ('*', 'Z', ('+', 'A', 'B')))


'X*Y*Z*(A+B)'

In [29]:
e = parse_arithmetic_expression("A-(B-C)")
print(e)
format_binop_expression(e)


('-', 'A', ('-', 'B', 'C'))


'A-(B-C)'

In [30]:
e = parse_arithmetic_expression("A+(B-C)")
print(e)
format_binop_expression(e)


('+', 'A', ('-', 'B', 'C'))


'A+B-C'